In [ ]:
import pygame
import time
import random

# Инициализация Pygame
pygame.init()

# Определение цветов
white = (255, 255, 255)
yellow = (255, 255, 102)
black = (0, 0, 0)
red = (213, 50, 80)
green = (0, 255, 0)
blue = (50, 153, 213)

# Размеры исходного игрового окна
original_dis_width = 600
original_dis_height = 400

# Текущие размеры игрового поля
dis_width = original_dis_width
dis_height = original_dis_height

# Флаг полноэкранного режима
is_fullscreen = False

# Создание окна игры
dis = pygame.display.set_mode((dis_width, dis_height))
pygame.display.set_caption('Snake Game')

# Таймер
clock = pygame.time.Clock()

# Настройки змейки
snake_block = 10
snake_speed = 7

# Шрифты
font_style = pygame.font.SysFont("bahnschrift", 25)
score_font = pygame.font.SysFont("comicsansms", 35)
menu_font = pygame.font.SysFont("comicsansms", 50)

# Предыдущие результаты
previous_scores = []

# Уровень скорости
speed_level = 1
speed_multiplier = 0.9

# Загрузка фонового изображения для игры
original_background_image = pygame.image.load('zmeika.png')
background_image = pygame.transform.scale(original_background_image, (dis_width, dis_height))

# Загрузка фона для стартового меню
menu_background_image = pygame.image.load('fon.png')
menu_background_image = pygame.transform.scale(menu_background_image, (dis_width, dis_height))

# Функции
def Your_score(score):
    value = score_font.render("Your Score: " + str(score), True, yellow)
    dis.blit(value, [0, 0])

def our_snake(snake_block, snake_list):
    for x in snake_list[:-1]:
        pygame.draw.rect(dis, blue, [x[0], x[1], snake_block, snake_block])
    pygame.draw.rect(dis, green, [snake_list[-1][0], snake_list[-1][1], snake_block, snake_block])  # Голова змейки

def message(msg, color, x, y):
    mesg = font_style.render(msg, True, color)
    dis.blit(mesg, [x, y])

def pause():
    paused = True
    message("Paused. Press SPACE to continue or Q to quit.", yellow, dis_width / 6, dis_height / 3)
    pygame.display.update()
    while paused:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    paused = False
                elif event.key == pygame.K_q:
                    pygame.quit()
                    quit()

# Глобальные переменные для хранения процентного положения еды
food_pos_x_percent = 0.5  # Процент от ширины экрана
food_pos_y_percent = 0.5  # Процент от высоты экрана

def reset_food_position():
    global foodx, foody, food_pos_x_percent, food_pos_y_percent
    foodx = round(random.randrange(0, dis_width - snake_block) / 10.0) * 10.0
    foody = round(random.randrange(0, dis_height - snake_block) / 10.0) * 10.0
    # Обновляем процентное положение еды
    food_pos_x_percent = foodx / dis_width
    food_pos_y_percent = foody / dis_height

def update_food_position():
    global foodx, foody, food_pos_x_percent, food_pos_y_percent
    # Обновляем абсолютное положение еды на основе процентного положения
    foodx = food_pos_x_percent * dis_width
    foody = food_pos_y_percent * dis_height
    # Обеспечиваем, что еда находится в пределах экрана
    foodx = min(foodx, dis_width - snake_block)
    foody = min(foody, dis_height - snake_block)

def toggle_fullscreen():
    global dis, is_fullscreen, dis_width, dis_height, background_image
    is_fullscreen = not is_fullscreen

    if is_fullscreen:
        dis = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
    else:
        dis = pygame.display.set_mode((original_dis_width, original_dis_height))

    dis_width, dis_height = dis.get_size()
    background_image = pygame.transform.scale(original_background_image, (dis_width, dis_height))
    update_food_position()

def update_game_state():
    global dis_width, dis_height, background_image
    dis_width, dis_height = dis.get_size()
    background_image = pygame.transform.scale(original_background_image, (dis_width, dis_height))
    reset_food_position()

def show_previous_scores():
    y = 30
    for score in previous_scores[-5:]:  # Show last 5 scores
        message(f"Score: {score[0]}, Time: {score[1]}s", yellow, 5, y)
        y += 30

def increase_speed():
    global snake_speed, speed_level
    snake_speed = int(snake_speed * speed_multiplier)
    speed_level += 1

def draw_button(text, x, y, width, height, ac, ic):
    mouse = pygame.mouse.get_pos()
    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(dis, ac, (x, y, width, height))
    else:
        pygame.draw.rect(dis, ic, (x, y, width, height))

    txt = menu_font.render(text, True, black)
    dis.blit(txt, (x + (width / 2 - txt.get_width() / 2), y + (height / 2 - txt.get_height() / 2)))

def main_menu():
    menu = True
    while menu:
        dis.blit(menu_background_image, [0, 0])
        draw_button("Правила", 150, 150, 300, 50, yellow, yellow)
        draw_button("Старт", 150, 250, 300, 50, green, green)
        draw_button("Счёт игрока", 150, 350, 300, 50, red, red)
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                if 150 <= mouse_pos[0] <= 450 and 150 <= mouse_pos[1] <= 200:
                    show_rules()
                if 150 <= mouse_pos[0] <= 450 and 250 <= mouse_pos[1] <= 300:
                    gameLoop()
                if 150 <= mouse_pos[0] <= 450 and 350 <= mouse_pos[1] <= 400:
                    show_scores()

def show_rules():
    rules = True
    while rules:
        dis.blit(menu_background_image, [0, 0])
        rule_text = font_style.render("Избегайте стен и своего хвоста.", True, white)
        dis.blit(rule_text, [dis_width / 6, dis_height / 4])
        draw_button("Назад", 150, 350, 300, 50, red, red)
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                if 150 <= mouse_pos[0] <= 450 and 350 <= mouse_pos[1] <= 400:
                    rules = False

def show_scores():
    scores = True
    while scores:
        dis.blit(menu_background_image, [0, 0])
        show_previous_scores()
        draw_button("Назад", 150, 350, 300, 50, red, red)
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                if 150 <= mouse_pos[0] <= 450 and 350 <= mouse_pos[1] <= 400:
                    scores = False

def gameLoop():
    global dis, background_image, snake_speed, dis_width, dis_height, foodx, foody
    game_over = False
    game_close = False

    x1 = dis_width / 2
    y1 = dis_height / 2

    x1_change = 0
    y1_change = 0

    snake_List = []
    Length_of_snake = 1

    start_time = time.time()

    reset_food_position()

    while not game_over:
        while game_close:
            dis.fill(black)
            message("You Lost! Press C-Play Again or Q-Quit", red, dis_width / 6, dis_height / 3)
            Your_score(Length_of_snake - 1)
            pygame.display.update()

            elapsed_time = int(time.time() - start_time)
            previous_scores.append((Length_of_snake - 1, elapsed_time))

            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:
                        game_over = True
                        game_close = False
                    if event.key == pygame.K_c:
                        gameLoop()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_a and x1_change == 0:
                    x1_change = -snake_block
                    y1_change = 0
                elif event.key == pygame.K_d and x1_change == 0:
                    x1_change = snake_block
                    y1_change = 0
                elif event.key == pygame.K_w and y1_change == 0:
                    y1_change = -snake_block
                    x1_change = 0
                elif event.key == pygame.K_s and y1_change == 0:
                    y1_change = snake_block
                    x1_change = 0
                elif event.key == pygame.K_SPACE:
                    pause()
                elif event.key == pygame.K_f:
                    toggle_fullscreen()
                    reset_food_position()

        x1 += x1_change
        y1 += y1_change

        if x1 >= dis_width:
            x1 = 0
        elif x1 < 0:
            x1 = dis_width - snake_block
        if y1 >= dis_height:
            y1 = 0
        elif y1 < 0:
            y1 = dis_height - snake_block

        dis.blit(background_image, [0, 0])
        pygame.draw.rect(dis, red, [foodx, foody, snake_block, snake_block])
        snake_Head = [x1, y1]
        snake_List.append(snake_Head)
        if len(snake_List) > Length_of_snake:
            del snake_List[0]

        for x in snake_List[:-1]:
            if x == snake_Head:
                game_close = True

        our_snake(snake_block, snake_List)
        Your_score(Length_of_snake - 1)

        pygame.display.update()

        if x1 == foodx and y1 == foody:
            reset_food_position()
            Length_of_snake += 1
            if Length_of_snake % 10 == 0:
                increase_speed()

        clock.tick(snake_speed)

    pygame.quit()
    quit()
    
main_menu()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
